## For species that are recoverred and assembled multiple times, what does the pangenome look like? What happens to it if we dereplicate
- Create pangenomes for 5 species using roary
- Look at genes that are core / cloud etc.
- Does dereplication remove these genes? If so how many?
- Cross with metagenomes  


From pig, species that were recovered most times:
- Lactobacillus amylovorus x
- Limosilactobacillus reuteri x
- Sodaliphilus sp004557565
- Bariatricus sp004560705
- Prevotella sp002251295
- Gemmiger qucibialis
- Floccifex porci
- Holdemanella porci

These species also have a variety of different strains in the pig mags. 

## For the comparing to metaGs, we need the dereplicated genes as well. 
- We have what genes are left if we dereplicate: How to use that? Ill do it by hand first, making a file for it

In [ ]:
# so for each fasta file we need to subset it..
# subset_drep_genes.smk

# genes to rankt also already works


In [ ]:
# srun
srun --account=ctbrowngrp -p bmh -J pan -t 2:00:00 -c 2 --mem=20gb --pty bash
mamba activate branchwater-skipmer

srun --account=ctbrowngrp -p bmh -J pan -t 1:00:00 -c 32 --mem=60gb --pty bash
srun --account=ctbrowngrp -p med2 -J panr -t 3:00:00 -c 32 --mem=60gb --pty bash



In [ ]:
# do some retries and a couple of genomes at a time for gbsketch
snakemake --use-conda --resources mem_mb=20000 --rerun-triggers mtime \
-c 2 --rerun-incomplete -k -s  snakefile_wip/1.collect_genomes.smk \

--config pang_org="Bariatricus sp004560705" pang_folder="bariatricus_sp004560705" -n
--config pang_org="Sodaliphilus sp004557565" pang_folder="sodaliphilus_sp004557565" -n
--config pang_org="Gemmiger qucibialis" pang_folder="g_qucibialis" -n
--config pang_org="Holdemanella porci" pang_folder="h_porci" -n
--config pang_org="Floccifex porci" pang_folder="f_porci" -n
--config pang_folder="prevotella_sp002251295" -n
pang_folder="VUNA01_sp002299625" -n
pang_folder="dysos_sp004558115" -n
pang_folder="fimisoma_sp002320005" -n
pang_folder="prevotella_sp002251295" -n
pang_folder="crypto_sp004552115" -n

pang_folder="collinsella_sp002391315" -n

# not in gtdb
--config pang_org="Cryptobacteroides sp004552115" pang_folder="crypto_sp004552115" -n
Dysosmobacter sp004558115
VUNA01_sp002299625



In [ ]:
# 2. Run prokka 
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 32 --rerun-incomplete -k -s  snakefile_wip/2.prokka.smk  -n


snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 32 --rerun-incomplete -k -s  snakefile_wip/2.prokka.smk \
--config pang_org="Sodaliphilus sp004557565" pang_folder="sodaliphilus_sp004557565" -n
--config pang_org="Lactobacillus amylovorus" pang_folder="l_amylovorus" -n

In [ ]:
# for species that are not in gtdb
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 32 --rerun-incomplete -k -s  snakefile_wip/2.prokka_noref.smk --config
pang_folder="VUNA01_sp002299625" -n
pang_folder="dysos_sp004558115" -n

pang_folder="crypto_sp004552115" -n


In [ ]:
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 32 --rerun-incomplete -k -s  snakefile_wip/4a.roary.smk \
--config pang_org="Limosilactobacillus reuteri" pang_folder="l_reuteri" -n


# 4a. Run roary snakemake with no drep
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 32 --rerun-incomplete -k -s  snakefile_wip/4a.roary.smk 

In [ ]:
# 5. Run genes to hashes (add shell as well), no cloud for reference strains??
snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 30 --rerun-incomplete -k -s  snakefile_wip/5.genes_to_hash.smk 

In [ ]:
# 6. Convert the fasta files to hashes, and predict proteins for hashes
snakemake --use-conda --resources mem_mb=40000 --rerun-triggers mtime \
-c 4 --rerun-incomplete -k -s  snakefile_wip/6.genes_torankt.smk 



In [ ]:
# 7. Run against the metagenomes
snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 20 --rerun-incomplete -k -s  7.compare_pang_to_metag.smk 

In [ ]:
# Want for each hash (only pig) in how many metags
# calc hash presence stays the same: Bc we want that dmp file
# then diff script for converting the dmp file to a csv: parse-dmp.singular.py

# in hash_calc_pig.smk
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 8 --rerun-incomplete -k -s snakefile_wip/hash_calc_pig.smk -n